In [139]:
# Generator of a BCH code over GF(q) with length n and designed min. distance d
n = 31
q = 2
d = 7

m = multiplicative_order(mod(q,n))
Fq = GF(q)
R.<x> = PolynomialRing(Fq)

F.<a> = Fq.extension(m)
b = a^((q^m-1)//n)

offset = 1
g = 1
for i in range(offset, offset+d-1):
    u = b^i
    min_poly = u.minimal_polynomial()
#     print("i={}, u={}, {}".format(i,u,min_poly))
    g = lcm(g, min_poly)

C = codes.CyclicCode(generator_pol = g, length=n)
print(C.generator_polynomial())
print("[{},{},{}]".format(C.length(), C.dimension(), C.minimum_distance()))

x^15 + x^11 + x^10 + x^9 + x^8 + x^7 + x^5 + x^3 + x^2 + x + 1
[31,16,7]


In [141]:
# set_random_seed(2)
def err_gen(n,t):
    e = zero_vector(Fq, n)
    for i in range(t):
        pos = randint(0, n-1)
        e[pos] = 1
    return e

t = (d-1) // 2

cword = C.random_element()
err = err_gen(n,t)
rword = cword + err

# print("cword:", cword)
print("error:", err)
print("rword:", rword)

error: (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)
rword: (1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1)


In [142]:
# rword = (1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1)
def evaluate(vec, ele):
    n = len(vec)
    return sum([vec[i]*ele^i for i in range(n)])

t = (d-1) // 2
s = [evaluate(rword, b^i) for i in range(1, 2*t+1)]
print("syndrome:", s)

smat = Matrix([s[j:j+t] for j in range(t)])

print("determinant:", smat.determinant())
vec = vector([-s[j] for j in range(t, 2*t)])
coeff_sigma = smat.solve_right(vec)

coeff_sigma


syndrome: [a^4 + a^3 + a^2 + 1, a^4 + a^2 + a, a^4 + a^3 + 1, a^4 + a^3 + 1, a^4 + a^2 + a, a^2 + a]
determinant: a^3


(a^4 + a + 1, a^4 + a^3 + 1, a^4 + a^3 + a^2 + 1)

In [143]:
for j in range(n):
    ele = b^j
    res = sum([coeff_sigma[t-i]*ele^i for i in range(1,t+1)]) + 1
    if res == 0:
        print("error location:", n-j)

error location: 24
error location: 19
error location: 5
